In [11]:
! pip list

Package         Version   Editable project location
--------------- --------- --------------------------------------------------------------------------------------------------------------------
diabetes        0.1.0
diabetesoop     0.1.0     /Users/dyada/Desktop/BSE/Computing for Data Science/homeworks/hm6/homework-6-computing-data-science/diabetesoop
dspipeline      0.1.0     /Users/dyada/Desktop/BSE/Computing for Data Science/homeworks/final/homework-final-computing-data-science/dspipeline
et-xmlfile      1.1.0
geographiclib   2.0
geopy           2.3.0
joblib          1.2.0
numpy           1.23.4
openpyxl        3.0.10
pandas          1.5.1
pip             22.3.1
python-dateutil 2.8.2
pytz            2022.6
scikit-learn    1.1.3
scipy           1.9.3
setuptools      65.5.1
six             1.16.0
sklearn         0.0.post1
threadpoolctl   3.1.0
wheel           0.37.1


In [1]:
import sys
package_path = '/Users/dyada/Desktop/BSE/Computing for Data Science/homeworks/final/homework-final-computing-data-science/dspipeline'
sys.path.append(package_path)

In [2]:
from dspipeline.data.load import Data
from dspipeline.processing.prep import DropNaNs, FillNaNs
from dspipeline.processing.transformer import Dummifier, Binarizer
from dspipeline.modeling.model import Model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [3]:
X_FEATURES = ['Tenure', 'CityTier', 'WarehouseToHome',]
Y_FEATURE = 'Churn'

#### a. Load the data.

In [4]:
data = Data()
X_y_train, X_y_test = data.from_excel('../data/E Commerce Dataset.xlsx', Y_FEATURE, 'E Comm')

print('X_y_train>>', X_y_train)
print('X_y_test>>', X_y_test)

X_y_train>>             Tenure PreferredLoginDevice  CityTier  WarehouseToHome  \
CustomerID                                                           
51788          9.0             Computer         3             16.0   
52148          6.0             Computer         3             13.0   
51718          8.0         Mobile Phone         1             15.0   
52293         15.0         Mobile Phone         3             11.0   
55579         12.0             Computer         1             13.0   
...            ...                  ...       ...              ...   
54067         12.0         Mobile Phone         1             16.0   
54564         15.0         Mobile Phone         3              7.0   
54771          1.0         Mobile Phone         1             12.0   
51385          6.0             Computer         3              6.0   
55141          1.0             Computer         3              9.0   

           PreferredPaymentMode  Gender  HourSpendOnApp  \
CustomerID        

In [5]:
X_y_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4504 entries, 51788 to 55141
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Tenure                       4291 non-null   float64
 1   PreferredLoginDevice         4504 non-null   object 
 2   CityTier                     4504 non-null   int64  
 3   WarehouseToHome              4298 non-null   float64
 4   PreferredPaymentMode         4504 non-null   object 
 5   Gender                       4504 non-null   object 
 6   HourSpendOnApp               4307 non-null   float64
 7   NumberOfDeviceRegistered     4504 non-null   int64  
 8   PreferedOrderCat             4504 non-null   object 
 9   SatisfactionScore            4504 non-null   int64  
 10  MaritalStatus                4504 non-null   object 
 11  NumberOfAddress              4504 non-null   int64  
 12  Complain                     4504 non-null   int64  
 13  OrderAmountHi

In [6]:
X_y_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1126 entries, 54007 to 51315
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Tenure                       1075 non-null   float64
 1   PreferredLoginDevice         1126 non-null   object 
 2   CityTier                     1126 non-null   int64  
 3   WarehouseToHome              1081 non-null   float64
 4   PreferredPaymentMode         1126 non-null   object 
 5   Gender                       1126 non-null   object 
 6   HourSpendOnApp               1068 non-null   float64
 7   NumberOfDeviceRegistered     1126 non-null   int64  
 8   PreferedOrderCat             1126 non-null   object 
 9   SatisfactionScore            1126 non-null   int64  
 10  MaritalStatus                1126 non-null   object 
 11  NumberOfAddress              1126 non-null   int64  
 12  Complain                     1126 non-null   int64  
 13  OrderAmountHi

## b. Remove those rows that contain NaN values in the columns: age, gender, ethnicity.

In [8]:
X_y_train = DropNaNs(X_y_train, ['age', 'gender', 'ethnicity']).drop()
X_y_test = DropNaNs(X_y_test, ['age', 'gender', 'ethnicity']).drop()

print('X_y_train>>', X_y_train.info())
print('X_y_test>>', X_y_test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7503 entries, 864 to 1708
Data columns (total 52 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   encounter_id                 7503 non-null   int64  
 1   hospital_id                  7503 non-null   int64  
 2   age                          7503 non-null   float64
 3   bmi                          6195 non-null   float64
 4   elective_surgery             7503 non-null   int64  
 5   ethnicity                    7503 non-null   object 
 6   gender                       7503 non-null   object 
 7   height                       7401 non-null   float64
 8   hospital_admit_source        6816 non-null   object 
 9   icu_admit_source             7484 non-null   object 
 10  icu_id                       7503 non-null   int64  
 11  icu_stay_type                7503 non-null   object 
 12  icu_type                     7503 non-null   object 
 13  pre_icu_los_days

## c. Fill NaN with the mean value of the column in the columns: height, weight.

In [9]:
print('X_y_train nans before filling>>>\n', X_y_train.isna().sum().T[['height', 'weight']])
print('X_y_test nans before filling>>>\n', X_y_test.isna().sum().T[['height', 'weight']])

X_y_train = FillNaNs(X_y_train, ['height', 'weight']).fill()
X_y_test = FillNaNs(X_y_test, ['height', 'weight']).fill()

print('X_y_train nans after filling>>>\n', X_y_train.isna().sum().T[['height', 'weight']])
print('X_y_test nans after filling>>>\n', X_y_test.isna().sum().T[['height', 'weight']])

X_y_train nans before filling>>>
 height     102
weight    1256
dtype: int64
X_y_test nans before filling>>>
 height     25
weight    300
dtype: int64
X_y_train nans after filling>>>
 height    0
weight    0
dtype: int64
X_y_test nans after filling>>>
 height    0
weight    0
dtype: int64


## d. Transformations

In [10]:
# dummify ethnicity
X_y_train = Dummifier(X_y_train, ['ethnicity']).generate()
X_y_test = Dummifier(X_y_test, ['ethnicity']).generate()

print('X_y_train>>', X_y_train.info())
print('X_y_test>>', X_y_test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7503 entries, 864 to 1708
Data columns (total 57 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   encounter_id                 7503 non-null   int64  
 1   hospital_id                  7503 non-null   int64  
 2   age                          7503 non-null   float64
 3   bmi                          6195 non-null   float64
 4   elective_surgery             7503 non-null   int64  
 5   gender                       7503 non-null   object 
 6   height                       7503 non-null   float64
 7   hospital_admit_source        6816 non-null   object 
 8   icu_admit_source             7484 non-null   object 
 9   icu_id                       7503 non-null   int64  
 10  icu_stay_type                7503 non-null   object 
 11  icu_type                     7503 non-null   object 
 12  pre_icu_los_days             7503 non-null   float64
 13  readmission_stat

In [11]:
# binarize gender
X_y_train = Binarizer(X_y_train, 'gender', 'M/F').generate()
X_y_test = Binarizer(X_y_test, 'gender', 'M/F').generate()

print('X_y_train>>', X_y_train.info())
print('X_y_test>>', X_y_test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7503 entries, 864 to 1708
Data columns (total 57 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   encounter_id                 7503 non-null   int64  
 1   hospital_id                  7503 non-null   int64  
 2   age                          7503 non-null   float64
 3   bmi                          6195 non-null   float64
 4   elective_surgery             7503 non-null   int64  
 5   height                       7503 non-null   float64
 6   hospital_admit_source        6816 non-null   object 
 7   icu_admit_source             7484 non-null   object 
 8   icu_id                       7503 non-null   int64  
 9   icu_stay_type                7503 non-null   object 
 10  icu_type                     7503 non-null   object 
 11  pre_icu_los_days             7503 non-null   float64
 12  readmission_status           7503 non-null   int64  
 13  weight          

### e. Model

In [12]:
# model_ = Model(feature_columns=X_FEATURES, target_column=Y_FEATURE, model=LogisticRegression, hyperparamaterers={'penalty': 'l2', 'solver': 'saga'})
model_ = Model(feature_columns=X_FEATURES, target_column=Y_FEATURE, model=RandomForestClassifier, hyperparamaterers={'n_estimators': 1000, 'criterion': 'gini'})

# f. Training & prediction

In [13]:
model_.train(X_y_train)
preds = model_.predict(X_y_test)
print('preds>>', preds)

Model has been fitted on training dataset!!
preds>> [[0.521   0.479  ]
 [0.993   0.007  ]
 [0.727   0.273  ]
 ...
 [0.75675 0.24325]
 [0.934   0.066  ]
 [0.965   0.035  ]]


# 7. roc_auc_score calculation

In [14]:
X_y_test['roc_auc_score'] = preds[:, 1]

print('AUC ROC score Train: ', roc_auc_score(X_y_test[Y_FEATURE], X_y_test['roc_auc_score']))

AUC ROC score Train:  0.62742663476874
